# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 3-1. Задача TSP: инкрементальные алгоритмы.

В этой задаче Вам предлагается сравнить алгоритмы Nearest Neighbour и Nearest Insertion в задаче Euclidean TSP.

**Даны:**
* Координаты точек плоскости, являющихся вершинами графа.

**Найти:**
* Перестановку вершин, задающих минимальный по длине гамильтонов цикл в графе.

Сделайте следующее:
* Скачайте файл [`tsp-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/tsp-instances.zip) и разархивируйте из него файлы со входами задачи TSP.
* Реализуйте функции `solve_tsp_nearest_neighbour` и `solve_tsp_nearest_insertion`.
* Запустите функцию `run_all()`, чтобы протестировать свой код и сравнить качество решений, получаемых Nearest Neighbour и Nearest Insertion. Сильно ли они отличаются? Запишите свои качественные выводы в 1-2 предложениях в последней ячейке ipynb-файла.

In [1]:
def read_tsp_instance(filename):
    with open(filename, 'r') as file:
        coordinates = []
        for line in file:
            line = line.strip().lower()
            if line.startswith('dimension'):
                coordinates = [(0,0)] * int(line.split()[-1])
            tokens = line.split()
            if len(tokens) == 3 and tokens[0].isdecimal():
                tokens = line.split()
                coordinates[int(tokens[0])-1] = tuple(map(float, tokens[1:]))
        return coordinates

from math import hypot

def euclidean_distance(point1, point2):
    return hypot(point1[0] - point2[0], point1[1] - point2[1])

def calculate_tour_length(instance, permutation):
    n = len(permutation)
    return sum(euclidean_distance(instance[permutation[i]], instance[permutation[(i+1) % n]]) for i in range(len(permutation)))

In [2]:
import random

In [3]:
def solve_tsp_nearest_neighbour(instance):
    path = [random.randrange(len(instance))]
    unvisited = set(range(len(instance))) - set(path)

    while unvisited:
        next_vertex = min(unvisited, key=lambda i: euclidean_distance(instance[i], instance[path[-1]]))
        unvisited.remove(next_vertex)
        path.append(next_vertex)

    return path

In [4]:
from itertools import combinations, product


def solve_tsp_nearest_insertion(instance):
    n_vertices = len(instance)
    starting_edge = min(combinations(range(n_vertices), r=2),
                        key=lambda p: euclidean_distance(instance[p[0]], instance[p[1]]))

    path = list(starting_edge)
    visited = set(path)
    unvisited = set(range(n_vertices)) - visited

    while len(visited) < n_vertices:
        next_vertex, _ = min(product(unvisited, visited),
                             key=lambda r: euclidean_distance(instance[r[0]], instance[r[1]]))

        def edge_replacement_cost(edge):
            _, (u, v) = edge
            return (euclidean_distance(instance[u], instance[next_vertex]) +
                    euclidean_distance(instance[next_vertex], instance[v]) -
                    euclidean_distance(instance[u], instance[v]))

        insertion_index, _ = min(enumerate(zip(path, path[1:] + [path[0]])), key=edge_replacement_cost)
        visited.add(next_vertex)
        unvisited.remove(next_vertex)
        path.insert(insertion_index + 1, next_vertex)

    return path

In [5]:
import time
from os.path import exists
from tabulate import tabulate


def run_all():
    instance_filenames = ['d198.tsp', 'd493.tsp', 'd657.tsp', 'd2103.tsp',
                          'pr107.tsp', 'pr152.tsp', 'pr439.tsp']
    table = []
    for filename in instance_filenames:
        if not exists(filename):
            print('File not found: “{}”. Skipping this instance.'.format(filename))
            continue
        print('Solving', filename)
        instance = read_tsp_instance(filename)
        time_start = time.monotonic()
        quality_nn = calculate_tour_length(instance, solve_tsp_nearest_neighbour(instance))
        time_nn = time.monotonic() - time_start
        time_start = time.monotonic()
        quality_ni = calculate_tour_length(instance, solve_tsp_nearest_insertion(instance))
        time_ni = time.monotonic() - time_start
        table.append([filename, f'{int(quality_nn)} ({time_nn:.2}s)', f'{int(quality_ni)} ({time_ni:.2}s)'])
    print()
    print(tabulate(table, headers=['Instance', 'NN', 'NI']))


In [6]:
run_all()

Solving d198.tsp
Solving d493.tsp
Solving d657.tsp
Solving d2103.tsp
Solving pr107.tsp
Solving pr152.tsp
Solving pr439.tsp

Instance    NN               NI
----------  ---------------  ----------------
d198.tsp    20391 (0.029s)   18050 (0.62s)
d493.tsp    43784 (0.054s)   42130 (8.6s)
d657.tsp    63486 (0.093s)   60081 (2.1e+01s)
d2103.tsp   94679 (1.0s)     85771 (7.9e+02s)
pr107.tsp   47033 (0.0026s)  53211 (0.096s)
pr152.tsp   85288 (0.0051s)  88661 (0.27s)
pr439.tsp   142214 (0.048s)  133705 (6.2s)


## Выводы



По этим графам нельзя сказать, что один из алгоритмов строго лучше, несмотря на разные теоритические оценки -- здесь они дают схожие результаты (отличие не больше 10%).
Увы, наивная реализация вставок работает намного дольше, чем алгоритм ближайшего соседа.